# Линейный SVM "своими руками"

## Генерируем обучающую и тестовую выборку для экспериментов

In [1]:
from sklearn.model_selection import train_test_split
from sklearn import datasets

X, y = datasets.make_classification(
    n_samples=10000, n_features=20, 
    n_classes=2, n_informative=20, 
    n_redundant=0,
    random_state=42
)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2,
    random_state=42
)

print(len(X), len(y))
print(len(X_train))

(10000, 10000)
8000


## Пишем свой класс для SVM

In [2]:
import numpy as np
from random import randint
import random


np.random.seed(42)
random.seed(42)


class MySVM(object):
    def __init__(self, C=10000):
        self.C = C # regularization constant

    # f(x) = <w,x> + w_0
    def f(self, x):
        return np.dot(self.w, x) + self.w0

    # a(x) = [f(x) > 0]
    def a(self, x):
        return 1 if self.f(x) > 0 else 0
    
    # predicting answers for X_test
    def predict(self, X_test):
        return np.array([model.a(x) for x in X_test])

    # l2-regularizator
    def reg(self):
        return 1.0 * sum(self.w ** 2) / (2.0 * self.C)

    # l2-regularizator derivative
    def der_reg(self):
        return self.w / self.C
        return 0.0

    # hinge loss
    def loss(self, x, answer):
        return max([0, 1 - answer * self.f(x)])

    # hinge loss derivative
    def der_loss(self, x, answer):
        return -1.0 if 1 - answer * self.f(x) > 0 else 0.0
        return 0.

    # fitting w and w_0 with SGD
    def fit(self, X_train, y_train):
        dim = len(X_train[0])
        self.w = np.random.rand(dim) # initial value for w
        self.w0 = np.random.randn() # initial value for w_0
        
        # 10000 steps is OK for this example
        # another variant is to continue iterations while error is still decreasing
        for k in range(10000):  
            
            # random example choise
            rand_index = randint(0, len(X_train) - 1) # generating random index
            x = X_train[rand_index]
            y = y_train[rand_index]

            # simple heuristic for step size
            step = 0.5 * 0.9 ** k

            # w update
            self.w = self.w - step * (y * x * self.der_loss(x,y) + self.der_reg())
            
            # w_0 update
            self.w0 = self.w0 - step * y * self.der_loss(x,y)

## Пробуем обучить наш классификатор и посмотреть на качество на тесте

In [34]:
model = MySVM(C=1)
model.fit(X_train, y_train)
print(model.w, model.w0)

(array([-4.96439223e-02,  2.90352244e-04, -1.21133053e-02,  3.25595710e-01,
        9.62349167e-02, -8.87314160e-02,  1.19134218e-02, -1.81370288e-02,
       -1.90391723e-01, -2.91690296e-01, -6.57670124e-02,  1.27703364e-01,
        2.77492229e-01,  1.00993162e-01, -2.69328584e-01,  6.66601364e-02,
        3.69483682e-01,  1.93414186e-01, -9.24380705e-02, -2.43316036e-01]), 0.45252773324226375)


In [35]:
predictions = model.predict(X_test)

In [36]:
print(predictions)

[0 1 1 ... 1 1 1]


In [37]:
print(y_test, len(y_test), sum(y_test))

(array([1, 0, 1, ..., 1, 0, 1]), 2000, 991)


In [38]:
print(len(predictions), sum(predictions))

(2000, 1340)


In [39]:
print(sum(predictions == y_test) / float(len(y_test)))

0.6615


## Задания:

### - Допишите недостающие функции в MySVM (производные и обновление весов)

### - Сравните качество с sklearn LinearSVC

In [40]:
from sklearn.svm import LinearSVC

In [41]:
model = LinearSVC(C=1)

model.fit(X_train, y_train)

LinearSVC(C=1, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [42]:
predictions = model.predict(X_test)

In [43]:
print(sum(predictions == y_test) / float(len(y_test)))

0.801
